# Plotting

A good way to illustrate the plotting possibilities is through a long list of demos.

Note that Osyris's plotting functions are wrapping Matplotlib's plotting functions,
and forwards most Matplotlib arguments to the underlying function.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

path = "osyrisdata/starformation"
data = osyris.Dataset(8, scale="au", path=path).load()
ind = np.argmax(data["hydro"]["density"])
center = data["amr"]["xyz"][ind.values]

## A 2D histogram of density vs magnetic field

By default, the `histogram2d` function will show a binned count of cells

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True)

Additional components can be overlayed using `layers`:

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"],
    {"data": data["hydro"]["mass"], "norm": "log"}, # layer 1
    {"data": data["amr"]["level"], "operation": "mean", "mode": "contour", "colors": "k"}, # layer 2
    loglog=True)

## Simple cut plane

Create a 2D gas density slice 2000 au wide through the plane normal to `z`,
with velocity vectors overlayed as arrows, once agains using `layers`:

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"}, # layer 1
             {"data": data["hydro"]["velocity"], "mode": "vec"}, # layer 2
             dx=2000 * osyris.units("au"),
             origin=center,
             direction="z")

## Cut plane at an arbitrary angle

The `plane` function will also accept a vector to define the normal direction to the plane.
In this example, we also change the colormap.

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=500 * osyris.units("au"),
             origin=center,
             direction=[-1, 1, 3],
             cmap="magma")

## Automatic “top/side” slice orientation according to angular momentum

Create a 2D slice of the logarithm of density 500 au wide using automatic orientation based on the angular momentum in the data.
This is useful for looking at disks.
Use the `"top"` direction for the slice to view the disk from above

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=500 * osyris.units("au"),
             origin=center,
             direction="top")

Use the `direction="side"` for the slice to view the disk from the side

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=500 * osyris.units("au"),
             origin=center,
             direction="side")

## Coloring vectors and streamlines

In this example, we represent the velocity field as vectors (left) and as streamlines (right).
The colours represent the magnitude of the velocity.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))
plt.subplots_adjust(wspace=0.35)

dx = 2000.0 * osyris.units("au")

osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec",
              "color": data["hydro"]["velocity"], "cmap": "spring_r",
              "cbar": False},
             dx=dx, origin=center, direction="z", ax=ax[0])
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "stream",
              "color": data["hydro"]["velocity"], "cmap": "jet",
              "cbar": False},
             dx=dx, origin=center, direction="z", ax=ax[1])

## Embedding plots in existing Matplotlib axes

In this example, we create two subplot axes with Matplotlib.

Next, we plot in the left panel the log of density as a coloured slice with velocity vectors.
The minimum and maximum of $\log(\rho)$ are forced to `-17` and `-11`.
We give the `plane` call the axes to use via the `ax` argument.
Next, we overlay some custom chosen density contours with different line styles and colours.

In the right panel, we plot a plane of temperature and overlay some lightgray contours showing the AMR levels.
We specify only integer contour levels.

In [ ]:
# Create figure
fig = plt.figure(figsize=(10, 3.5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
plt.subplots_adjust(wspace=0.5)

# Define region to plot
dx = 2000.0 * osyris.units("au")

# Left plot: coloured density slice with overlayed contours
osyris.plane({"data": data["hydro"]["density"], "norm": "log",
                               "vmin": 1.0e-17,
                               "vmax": 1.0e-11},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             {"data": data["hydro"]["density"], "mode": "contour",
                  "levels": [1.0e-17,1.0e-16,1.0e-12],
                  "colors": ('yellow','k',"lime"),
                  "linewidths": [2,5,2],
                  "linestyles": ["solid","dashed","solid"],
                  "cmap": None,
                  "labels": False},
             dx=dx,
             origin=center,
             direction="z", ax=ax1)

osyris.plane({"data": data["hydro"]["temperature"], "norm": "log", "mode": "contourf",
              "levels": np.logspace(0.9, 2, 21), "cmap": "hot"},
             {"data": data["amr"]["level"], "mode": "contour", "colors": "w", "levels": [6, 7, 8]},
             dx=dx,
             origin=center,
             direction="z", ax=ax2)

## Plot only a subset of cells belonging to a disk

In this example, we select cells according to their density and plot only those.
This is done by creating a new field and using Numpy's `masked_where` function.
To combine more than one selection criteria, we use Numpy's `logical_or` function.

This is useful for plotting disks around protostars, for example.
Here we select the cells with a density in the range
$5 \times 10^{-14}~\text{g cm}^{-3} < \rho < 5 \times 10^{-12}~\text{g cm}^{-3}$.

In [ ]:
data["hydro"]["disk_density"] = osyris.Array(
    np.ma.masked_where(np.logical_or(
        data["hydro"]["density"].values < 5.0e-14,
        data["hydro"]["density"].values > 5.0e-12),
                       data["hydro"]["density"].values),
                       unit=data["hydro"]["density"].unit)
osyris.plane(data["hydro"]["disk_density"], dx=500 * osyris.units("au"),
             norm="log", origin=center, mode="image")

Computing the disk mass can be achieved via

In [ ]:
np.sum(data["hydro"]["disk_density"]*(data["amr"]["dx"]**3)).to("msun")

## Difference between two snapshots

Here, we want to make a map of the difference in density between two snapshots.
Because we do not necessarily have the same number of cells at the same place,
we first have to make uniform 2D maps using the `plane` function,
which we can then directly compare.

The `plane` function actually returns an object that contains the raw data that was used to create the figure.
By using the `plot=False` argument, we can silence the figure generation, and use the data in a custom figure.

**Note:** For this to make sense, the two outputs have to be centered around the same center point.

In [ ]:
# Read data from an earlier snapshot
data_old = osyris.Dataset(5, scale="au", path=path).load()

In [ ]:
dx = 2000 * osyris.units("au")
# Extract density slices by copying data into structures
plane1 = osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
                      dx=dx, origin=center, direction="z", plot=False)

plane2 = osyris.plane({"data": data_old["hydro"]["density"], "norm": "log"},
                      dx=dx, origin=center, direction="z", plot=False)

# Get x,y coordinates
x = plane1.x
y = plane1.y

# Density difference
rho1 = np.log10(plane1.layers[0]["data"])
rho2 = np.log10(plane2.layers[0]["data"])
diff = (rho1 - rho2) / rho2

# Create figure
fig, ax = plt.subplots()
cf = ax.contourf(x, y , diff, cmap='RdBu',
                 levels=np.linspace(-0.12,0.12,31))
ax.set_aspect("equal")
cb = plt.colorbar(cf, ax=ax)
cb.ax.set_ylabel("Relative difference")

## Slice above the origin

We want to plot a slice of density but through a point which is 20 AU above the centre of the domain,
defined as the cell with the highest density.

In [ ]:
origin = center.copy()
origin.values[-1] += 20
osyris.plane({"data": data["hydro"]["density"], "norm": "log"},
             {"data": data["hydro"]["velocity"], "mode": "vec"},
             dx=500 * osyris.units("au"),
             origin=origin,
             direction="z")

## Scatter plot

Make a scatter plot of density vs temperature, with only 1 in 1000 cells.
Colour the dots according to the magnitude of the gas velocity.

In [ ]:
step = 100
osyris.scatter(data["hydro"]["density"][::step],
               data["hydro"]["velocity"][::step],
               c=data["hydro"]["thermal_pressure"][::step],
               norm="log", loglog=True)

## Sink particles

Some star formation simulations contain sink particles,
and these can be represented as a scatter layer on a plane plot. 

In [ ]:
osyris.plane({"data": data["hydro"]["density"], "norm": "log"}, # layer 1
             {"data": data["sink"]["xyz"], "mode": "scatter", "c": "white"}, # layer 2
             dx=2000 * osyris.units("au"),
             origin=center,
             direction="z")

## 2D data

2D data can be loaded and viewed just like 3D data.

In [ ]:
data2d = osyris.Dataset(2, scale="cm", path="osyrisdata/sedov").load()
osyris.plane(data2d["hydro"]["density"], norm="log", cmap="jet")

## 1D histograms

The `histogram1d` function provides a simple way to quickly make 1d histogram plots:

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True)

The bin edges can be specified using the `bins` parameter,
which can either be an integer number or an array (similarly to Numpy's `bins` argument):

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True,
                   bins=np.logspace(-18., -13., 10),
                   color='m')

Multiple histograms can be over-plotted on the same axes by passing multiple layers:

In [ ]:
bins = np.linspace(-0.15, 0.15, 40)
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].z, "alpha": 0.5},
                   logy=True, bins=bins)

## Line integral convolution vector field visualizations

Osyris now supports line integral convolution (LIC) visualizations of vector fields. This visualization method, although computationally intensive, offers an excellent representation of the vector field (especially near discontinuities) provided we have enough resolution.

**Note:** This feature requires the [lic](https://pypi.org/project/lic/) package to be installed.

Below is an example of a LIC of the velocity vector field:

In [ ]:
osyris.plane({"data": data["hydro"]["velocity"], "mode": "lic"},
             dx=2000 * osyris.units("au"),
             origin=center,
             direction="z", resolution=1000)

The LIC can be colored by a scalar field using the `color` keyword, and the length of the lines of the LIC can also be modified using the `length` keyword (whose default value is 30):

In [ ]:
osyris.plane({"data": data["hydro"]["velocity"], "mode": "lic",
              "color": data["hydro"]["density"], "norm": "log", "length":30},
             dx=2000 * osyris.units("au"),
             origin=center,
             direction="z", resolution=1000)

Overlaying this plot with velocity arrows allows us to retrieve some information on the magnitude of the vector field.